In [1]:
from utils import get_spark_session
import time
from data_generator import gen_bulk_data

spark = get_spark_session("iceberg_table_partitioning")
df_test = gen_bulk_data(spark, 10**4)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2f60b165-2794-4a64-ba8d-d780dcf93d1f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.95.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found softwa

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- salary: double (nullable = true)
 |-- last_purchase: timestamp (nullable = true)
 |-- last_purchase_date: string (nullable = true)



In [6]:
TABLE_NAME = "nessie.dev.partitioning"

df_test.createOrReplaceTempView("df_test")
spark.sql(f"""
CREATE OR REPLACE TABLE {TABLE_NAME} USING ICEBERG PARTITIONED BY (age) AS SELECT * FROM df_test ORDER BY age
""")

spark.table(TABLE_NAME).show()

+-------+--------------------+---+-------+-------------------+------------------+
|     id|                name|age| salary|      last_purchase|last_purchase_date|
+-------+--------------------+---+-------+-------------------+------------------+
|9928165|          Gabriel Sá|  0|1011.11|1983-06-03 14:02:36|        1983-06-03|
|9638342|      Manuella Porto|  0| 981.63|2013-01-09 03:03:25|        2013-01-09|
| 294849|       Bryan Pereira|  0|1000.09|1999-03-13 21:29:49|        1999-03-13|
|3231746|       Evelyn Novaes|  0| 989.01|1981-09-30 06:29:19|        1981-09-30|
|5698933|    Benjamim Camargo|  0| 989.02|1989-08-13 04:48:17|        1989-08-13|
|9952876|  Dra. Sabrina Jesus|  0|1000.37|1984-11-05 09:35:27|        1984-11-05|
|9910922|  Sra. Isabela Rocha|  0| 998.74|2018-03-05 14:37:04|        2018-03-05|
|5217730|      Olivia Fonseca|  0|1011.34|1971-07-08 05:18:33|        1971-07-08|
|4456269|     Ayla Nascimento|  0|1001.05|2018-01-09 22:22:08|        2018-01-09|
|1561052|Srta. M

In [10]:
spark.sql(f"ALTER TABLE {TABLE_NAME} ADD PARTITION FIELD truncate(1, name)").show()

spark.sql(f"CALL nessie.system.rewrite_data_files('{TABLE_NAME}')").show()

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|                       101|                  1976|               492765|                      0|
+--------------------------+----------------------+---------------------+-----------------------+



In [11]:
spark.sql(f"""
    SELECT * FROM {TABLE_NAME}.history;
""").show()


+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-09-29 22:52:...|8248509610644572393|               NULL|              false|
|2024-09-29 22:55:...|5582960184519021905|               NULL|               true|
|2024-09-29 23:02:...|8285212853883752419|5582960184519021905|               true|
+--------------------+-------------------+-------------------+-------------------+



24/09/30 00:44:39 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/09/30 00:44:39 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:981)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [ ]:
spark.stop()